In [54]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType,StructField,StringType,ArrayType

In [55]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Chapter 7 - Aggregations") \
    .getOrCreate()

In [56]:
spark

In [57]:
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("online-retail-dataset.csv")\
.coalesce(5)

In [58]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [59]:
df.count()

541909

In [60]:
df.cache()
df.createOrReplaceTempView("dfTable")

In [61]:
df.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

#### **COUNT**
La primera función que trataremos será **COUNT**, excepto que en este ejemplo funcionara como una transformación y no como una acción.
* Especificar una columna para contar COUNT(“COLUMN_NAME”)
* Todas las columnas usando COUNT(*) 
* COUNT(1) para contar cada fila.

In [62]:
df.select(count("StockCode")).show()
#df.select(count("*")).show()
#df.select(count(col("StockCode"))).show()
#df.select(count(lit(1))).show()
#spark.sql("SELECT COUNT(1) FROM dfTable").show()

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



#### **COUNTDISTINCT**
A veces el numero total no es relevante, podria serlo la cantidad de grupos unicos. Esta funcion es mas relevante para columnas individuales

In [63]:
df.select(countDistinct("StockCode")).show()
#spark.sql("SELECT COUNT(DISTINCT *) FROM dftable").show()
#df.groupBy("StockCode").agg(count("StockCode").alias("count")).filter(col("count") > 1).show()
#spark.sql("SELECT StockCode, count(StockCode) FROM dftable GROUP BY StockCode HAVING COUNT(StockCode) > 1").show()


+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



#### **APPROX_COUNT_DISTINCT**
A menudo, nos encontramos trabajando con grandes conjuntos de datos y la funcion *COUNT DISTINCT* es irrelevante.  
Hay momentos en que una aproximación con cierto grado de precisión funcionará bien, y
para eso, puedes usar la función *APPROX_COUNT_DISTINCT*  

Notarás que **approx_count_distinct** tomó otro parámetro con el que puedes especificar el error de estimación máximo permitido. En este caso, especificamos un error bastante grande y por lo tanto, recibe una respuesta que está bastante lejos pero se completa más rápido que countDistinct.  


Verá ganancias de rendimiento mucho mayores con conjuntos de datos más grandes.

In [64]:
df.select(approx_count_distinct("StockCode", 0.1)).show()
spark.sql("SELECT approx_count_distinct(StockCode, 0.1) FROM DFTABLE").show()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



#### **FIRST AND LAST**

Puede obtener el primer y el último valor de una columna de un DataFrame haciendo uso de las funciones **FIRSTS** y **LAST**

In [65]:
df.select(first("StockCode"), last("StockCode")).show()
#spark.sql("SELECT first(StockCode), last(StockCode) FROM dfTable").show()
#df.orderBy(desc("StockCode")).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          22138|
+----------------+---------------+



#### **MIN AND MAX**

Para extraer el minimo y maximo valor de un Dataframe podemos hacer uso de las funciones **MIN** y **MAX**

In [66]:
df.select(min("Quantity"), max("Quantity")).show()
#spark.sql("SELECT min(Quantity), max(Quantity) FROM dfTable").show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



#### **SUM**
Otra tarea simple es sumar todos los valores en una columna haciendo uso de la funcion **SUM**

In [67]:
df.select(sum("Quantity")).show()
#spark.sql("SELECT sum(Quantity) FROM dfTable").show()

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



### **SUMDISTINCT**
Además de sumar un total, también puede sumar un conjunto de valores unicos

In [68]:
df.select(sumDistinct("Quantity")).show()
#spark.sql("SELECT SUM(DISTINCT(Quantity)) FROM dfTable").show()

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



/usr/local/spark/python/pyspark/sql/functions.py:214: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
  warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)


#### **AVG**
Aunque puede calcular el promedio dividiendo la suma por el conteo, Spark proporciona una forma más fácil de obtener ese valor a través de las funciones **AVG** o **MEAN**. En este ejemplo, usamos alias para reutilizar más fácilmente estas columnas más adelante.  

También puede promediar todos los valores distintos especificando **distinct**. De hecho, la mayoría de las funciones agregadas
admite hacerlo solo en valores distintos.

In [69]:
df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases"))\
.selectExpr(
"total_purchases/total_transactions",
"avg_purchases",
"mean_purchases").show()

+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|   avg_purchases|  mean_purchases|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



#### **Aggregating to Complex Types**
En Spark, puede realizar agregaciones no solo de valores numéricos usando fórmulas, también puede realizarlas en tipos complejos. Por ejemplo, podemos recopilar una lista de valores presentes en una columna determinada o solo los valores únicos.  

Puede usar esto para realizar acceso programático en la canalización o pasar la colección completa en una función definida por el usuario (UDF).  
*collect_list* - Genera un orden en la Matriz pero no elimina los datos duplicados  
*collect_set* - No puiede guardar un orden existente pero si elimina los elementos duplicados

In [70]:
df.agg(collect_set("Country"), collect_list("Country")).show()
#df.groupBy("InvoiceNo").agg(collect_list("Country")).show()
#df.groupBy("InvoiceNo").agg(array_distinct(collect_list("Country"))).show()
#spark.sql("SELECT collect_set(Country), collect_set(Country) FROM dfTable").show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



### **Grouping**

Hasta ahora, solo hemos realizado agregaciones a nivel de DataFrame. Una tarea más común es realizar cálculos basados en grupos de datos. Esto generalmente se hace en datos categóricos para los cuales agrupamos nuestros datos en una columna y realizamos algunos cálculos en las otras columnas de ese grupo.  

El primero será un conteo, tal como lo hicimos antes. Agruparemos por cada número de factura (InvoiceNo) y obtendremos el recuento de artículos en esa factura. Tenga en cuenta que esto devuelve otro DataFrame.  

Esta agrupación la hacemos en dos fases. Primero especificamos la(s) columna(s) en las que nos gustaría agrupar, y luego especificamos la(s) agregación(es).  

El primer paso devuelve un RelationalGroupedDataset y el segundo paso devuelve un DataFrame.  

Como se mencionó, podemos especificar cualquier número de columnas en las que queremos agrupar.  

In [71]:
df.groupBy("InvoiceNo", "CustomerId").count().show()
#spark.sql("SELECT InvoiceNo, CustomerId, count(*) FROM dfTable GROUP BY InvoiceNo, CustomerId").show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536846|     14573|   76|
|   537026|     12395|   12|
|   537883|     14437|    5|
|   538068|     17978|   12|
|   538279|     14952|    7|
|   538800|     16458|   10|
|   538942|     17346|   12|
|  C539947|     13854|    1|
|   540096|     13253|   16|
|   540530|     14755|   27|
|   541225|     14099|   19|
|   541978|     13551|    4|
|   542093|     17677|   16|
|   543188|     12567|   63|
|   543590|     17377|   19|
|  C543757|     13115|    1|
|  C544318|     12989|    1|
|   544578|     12365|    1|
|   545165|     16339|   20|
|   545289|     14732|   30|
+---------+----------+-----+
only showing top 20 rows



#### **Grouping with Expressions**

Esto le permite pasar expresiones arbitrarias que solo necesitan tener alguna agregación especificada. Incluso puede hacer cosas como alias de una columna después de transformarla para su uso posterior en su flujo de datos

In [72]:
df.groupBy("InvoiceNo").agg(
count("Quantity").alias("Count_Quantity"),
expr("count(Quantity)").alias("Expression_Count_Quantity")).show()

+---------+--------------+-------------------------+
|InvoiceNo|Count_Quantity|Expression_Count_Quantity|
+---------+--------------+-------------------------+
|   536596|             6|                        6|
|   536938|            14|                       14|
|   537252|             1|                        1|
|   537691|            20|                       20|
|   538041|             1|                        1|
|   538184|            26|                       26|
|   538517|            53|                       53|
|   538879|            19|                       19|
|   539275|             6|                        6|
|   539630|            12|                       12|
|   540499|            24|                       24|
|   540540|            22|                       22|
|  C540850|             1|                        1|
|   540976|            48|                       48|
|   541432|             4|                        4|
|   541518|           101|                    

#### **Grouping with Maps**
A veces, puede ser más fácil especificar sus transformaciones como una serie de mapas para los cuales la clave es la columna y el valor es la función de agregación (como una cadena). También puede reutilizar varios nombres de columna si los especifica en línea.

In [76]:
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)")).show()
#spark.sql("SELECT InvoiceNo, avg(Quantity), stddev_pop(Quantity) FROM dfTable GROUP BY InvoiceNo").show()

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536938|33.142857142857146|  20.698023172885524|
|   537252|              31.0|                 0.0|
|   537691|              8.15|   5.597097462078001|
|   538041|              30.0|                 0.0|
|   538184|12.076923076923077|   8.142590198943392|
|   538517|3.0377358490566038|  2.3946659604837897|
|   538879|21.157894736842106|  11.811070444356483|
|   539275|              26.0|  12.806248474865697|
|   539630|20.333333333333332|  10.225241100118645|
|   540499|              3.75|  2.6653642652865788|
|   540540|2.1363636363636362|  1.0572457590557278|
|  C540850|              -1.0|                 0.0|
|   540976|10.520833333333334|   6.496760677872902|
|   541432|             12.25|  10.825317547305483|
|   541518| 23.10891089108911|  20.550782784878713|
|   541783|1

#### **PIVOT**
PySpark SQL proporciona la función para rotar los datos de una columna en varias columnas. Es una agregación donde uno de los valores de las columnas de agrupación se transpone en columnas individuales con datos distintos. En el siguiente ejemplo para obtener la cantidad total exportada a cada país de cada producto se puede hacer de la siguiente manera:

In [82]:
data = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"), \
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"), \
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"), \
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico"),\
      ("Banana",3000,"Canada"),("Carrots",3000,"Canada"),("Beans",3000,"Mexico")]

columns= ["Product","Amount","Country"]
df_pivot = spark.createDataFrame(data = data, schema = columns)
df_pivot.printSchema()
df_pivot.show(truncate=False)

root
 |-- Product: string (nullable = true)
 |-- Amount: long (nullable = true)
 |-- Country: string (nullable = true)

+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
|Banana |1000  |USA    |
|Carrots|1500  |USA    |
|Beans  |1600  |USA    |
|Orange |2000  |USA    |
|Orange |2000  |USA    |
|Banana |400   |China  |
|Carrots|1200  |China  |
|Beans  |1500  |China  |
|Orange |4000  |China  |
|Banana |2000  |Canada |
|Carrots|2000  |Canada |
|Beans  |2000  |Mexico |
|Banana |3000  |Canada |
|Carrots|3000  |Canada |
|Beans  |3000  |Mexico |
+-------+------+-------+



In [83]:
pivotDF = df_pivot.groupBy("Product").pivot("Country").sum("Amount")
pivotDF.printSchema()
pivotDF.show(truncate=False)

root
 |-- Product: string (nullable = true)
 |-- Canada: long (nullable = true)
 |-- China: long (nullable = true)
 |-- Mexico: long (nullable = true)
 |-- USA: long (nullable = true)

+-------+------+-----+------+----+
|Product|Canada|China|Mexico|USA |
+-------+------+-----+------+----+
|Orange |null  |4000 |null  |4000|
|Beans  |null  |1500 |5000  |1600|
|Banana |5000  |400  |null  |1000|
|Carrots|5000  |1200 |null  |1500|
+-------+------+-----+------+----+



#### **Window Functions**
También puede usar funciones de ventana para realizar algunas agregaciones únicas, ya sea calculando alguna agregación en una "ventana" específica de datos, que define usando una referencia a la
datos actuales. Esta especificación de ventana determina qué filas se pasarán a esta función.

Ahora bien, esto es un poco abstracto y probablemente similar a un grupo estándar, así que diferenciémoslos un poco más.

Un groupby toma datos, y cada fila puede ir solo a una agrupación. Una función de ventana calcula un valor de retorno para cada fila de entrada de una tabla en función de un grupo de filas, denominado Frame.

Cada fila puede caer en uno o más Frames. Un caso de uso común es echar un vistazo a un promedio móvil de algún valor para el cual cada fila representa un día. Si hiciera esto, cada fila terminaría en siete marcos diferentes. Cubriremos la definición de Frame un poco más adelante, pero para su referencia, Spark admite tres tipos de funciones de ventana: funciones de clasificación, funciones analíticas y funciones agregadas.  

Para demostrarlo, agregaremos una columna de fecha que convertirá la fecha de su factura en una columna que contiene solo información de fecha (No tendra informacion de la hora):

In [77]:
simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
  )
 
columns= ["employee_name", "department", "salary"]
df_window_function = spark.createDataFrame(data = simpleData, schema = columns)

windowSpec  = Window.partitionBy("department").orderBy("salary")
windowSpecAgg  = Window.partitionBy("department")
from pyspark.sql.functions import col,avg,sum,min,max,row_number 
df_window_function.withColumn("row",row_number().over(windowSpec)) \
  .withColumn("avg", avg(col("salary")).over(windowSpecAgg)) \
  .withColumn("sum", sum(col("salary")).over(windowSpecAgg)) \
  .withColumn("min", min(col("salary")).over(windowSpecAgg)) \
  .withColumn("max", max(col("salary")).over(windowSpecAgg)) \
  .where(col("row")==1).select("department","avg","sum","min","max") \
  .show()


+----------+------+-----+----+----+
|department|   avg|  sum| min| max|
+----------+------+-----+----+----+
|   Finance|3400.0|10200|3000|3900|
| Marketing|2500.0| 5000|2000|3000|
|     Sales|3760.0|18800|3000|4600|
+----------+------+-----+----+----+



In [ ]:
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")

dfWithDate.select("date").show()

El primer paso para una Window Function es crear una especificación de ventana. Es solo un concepto similar que describe cómo dividiremos nuestro grupo. El orden determina el orden dentro de una partición determinada y, por último, la especificación del Frame (la instrucción rowsBetween) establece qué filas se incluirán en el marco en función de su referencia a la fila de entrada actual. En el siguiente ejemplo, observamos todas las filas anteriores hasta la fila actual.  

**Window.unboundedPreceding** denota la primera fila de la partición y **Window.unboundedFollowing** denota la última fila de la partición.

**Window.currentRow** para especificar un valor actual en una fila.

In [ ]:
from pyspark.sql.window import Window
windowSpec = Window\
.partitionBy("CustomerId")\
.orderBy(desc("Quantity"))\
.rowsBetween(Window.unboundedPreceding, Window.currentRow)

Ahora queremos usar una función de agregación para obtener más información sobre cada cliente específico. Un ejemplo podría ser establecer la cantidad máxima de compra en todo momento. Para responder a esto, usamos las mismas funciones de agregación que vimos anteriormente al pasar un nombre de columna o una expresión.

Además, indicamos la especificación de la ventana que define a qué marcos de datos se aplicará esta función:

In [ ]:
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

In [ ]:
range_between_df = df.withColumn("max_quantity", max("Quantity").over(windowSpec))

range_between_df.show()

+---------+---------+--------------------+--------+----------------+---------+----------+--------------+------------+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|max_quantity|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+------------+
|   542504|    37413|                null|    5568| 1/28/2011 12:03|      0.0|      null|United Kingdom|        5568|
|   556231|   85123A|                   ?|    4000|  6/9/2011 15:04|      0.0|      null|United Kingdom|        5568|
|   560040|    23343| came coded as 20713|    3100| 7/14/2011 14:28|      0.0|      null|United Kingdom|        5568|
|   546139|    84988|                   ?|    3000|  3/9/2011 16:35|      0.0|      null|United Kingdom|        5568|
|   542505|   79063D|                null|    2560| 1/28/2011 12:04|      0.0|      null|United Kingdom|        5568|
|   574941|    22197|      POPCORN HOLDER|    1820| 11/7

Notará que esto devuelve una columna (o expresiones). Ahora podemos usar esto en una declaración de selección (SELECT) de un DataFrame. Sin embargo, antes de hacerlo, crearemos el rango de cantidad de compra. Para hacer eso, usamos la función dense_rank para determinar qué fecha tuvo la cantidad máxima de compra para cada cliente. Usamos dense_rank en lugar de rank para evitar brechas en la secuencia de clasificación cuando hay valores empatados (o en nuestro caso, filas duplicadas):

In [ ]:
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

Esto también devuelve una columna que podemos usar en declaraciones de selección. Ahora podemos realizar una selección para ver los valores de ventana calculados.

In [ ]:
dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
.select(
col("CustomerId"),
col("date"),
col("Quantity"),
purchaseRank.alias("quantityRank"),
purchaseDenseRank.alias("quantityDenseRank"),
maxPurchaseQuantity.alias("maxPurchaseQuantity")).show()

In [ ]:
spark.sql("SELECT CustomerId, date, Quantity,\
rank(Quantity) OVER (PARTITION BY CustomerId, date\
ORDER BY Quantity DESC NULLS LAST\
ROWS BETWEEN\
UNBOUNDED PRECEDING AND\
CURRENT ROW) as rank,\
dense_rank(Quantity) OVER (PARTITION BY CustomerId, date\
ORDER BY Quantity DESC NULLS LAST\
ROWS BETWEEN\
UNBOUNDED PRECEDING AND\
CURRENT ROW) as dRank,\
max(Quantity) OVER (PARTITION BY CustomerId, date\
ORDER BY Quantity DESC NULLS LAST\
ROWS BETWEEN\
UNBOUNDED PRECEDING AND\
CURRENT ROW) as maxPurchase\
FROM dfWithDate WHERE CustomerId IS NOT NULL ORDER BY CustomerId").show()